In [1]:
!pip install ntscraper

In [2]:
from ntscraper import Nitter

In [3]:
scraper = Nitter(log_level = 1,skip_instance_check = False)

Testing instances: 100%|██████████| 77/77 [01:19<00:00,  1.03s/it]


In [19]:
scraper.get_tweets("3rdEyeDude",mode="user",number=100)

INFO:root:No instance specified, using random instance https://nitter.privacydev.net


{'tweets': [], 'threads': []}